# Tech Challenge Fase 2

# Aluno
Klauber Lage - RM358972

Link vídeo no Youtube: https://www.youtube.com/watch?v=m9NDA8SMadw&ab_channel=KlauberLage

Link repositório no GitHub: https://github.com/klauberfreitas/fiap/tree/main/Fase%201/Tech%20Challenge

# Tarefas


### Referência:
[Documento PDF do Desafio](POSTECH-TechChallenge-IADT-Fase1.pdf)

---



# Relatório 

# 0 - Instalação de dependências

In [1]:
# ! pip install numpy random

In [2]:
# Importando o pandas 
import random

import numpy as np

In [3]:
# Definição das cidades e suas coordenadas
cities = {
    'A': (0, 0),
    'B': (1, 3),
    'C': (4, 3),
    'D': (6, 1),
    'E': (3, 0)
}

city_names = list(cities.keys())
num_cities = len(city_names)

# Função para calcular a distância entre duas cidades
def distance(city1, city2):
    x1, y1 = cities[city1]
    x2, y2 = cities[city2]
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Função de aptidão: calcular a distância total da rota
def fitness_function(route):
    total_distance = 0
    for i in range(len(route)):
        total_distance += distance(route[i], route[(i + 1) % num_cities])
    return -total_distance  # Queremos minimizar a distância

# Configurações do Algoritmo Genético
population_size = 100
num_generations = 50
mutation_rate = 0.01

# Inicialização da população: permutações aleatórias das cidades
population = [random.sample(city_names, num_cities) for _ in range(population_size)]

# Evolução da população
for generation in range(num_generations):
    # Avaliação da aptidão
    fitness_values = [fitness_function(individual) for individual in population]
    
    # Seleção dos melhores indivíduos
    selected_individuals = [population[i] for i in np.argsort(fitness_values)[-population_size//2:]]
    
    # Cruzamento e mutação
    new_population = []
    for _ in range(population_size):
        parent1, parent2 = random.sample(selected_individuals, 2)
        crossover_point = random.randint(1, num_cities-1)
        child = parent1[:crossover_point] + [city for city in parent2 if city not in parent1[:crossover_point]]
        
        # Mutação
        if random.random() < mutation_rate:
            swap_idx1, swap_idx2 = random.sample(range(num_cities), 2)
            child[swap_idx1], child[swap_idx2] = child[swap_idx2], child[swap_idx1]
        
        new_population.append(child)
    
    population = new_population

# Melhor solução encontrada
best_solution = max(population, key=fitness_function)
print("Melhor solução:", best_solution)
print("Distância total:", -fitness_function(best_solution))

Melhor solução: ['A', 'E', 'D', 'C', 'B']
Distância total: 15.15298244508295


In [4]:
! pip install pygame

In [5]:

import os
    
os.chdir(os.path.expanduser(".././assets/"))

In [6]:
def tournament_selection(population, population_fitness, tournament_size=3):
    selected_parents = []
    
    for _ in range(2):  # Selecionar dois pais
        tournament = random.sample(list(zip(population, population_fitness)), tournament_size)
        # Selecionar o indivíduo com o melhor fitness no torneio
        winner = min(tournament, key=lambda x: x[1])
        selected_parents.append(winner[0])
    
    return selected_parents

In [7]:
import json
def save_best_individual(best_individual, best_fitness, filename="best_individual.json"):
    print("saved")
    data = {
        "best_individual": best_individual,
        "best_fitness": best_fitness
    }
    with open(filename, 'w') as f:
        json.dump(data, f)


In [8]:

def load_best_individual(filename="best_individual.json"):
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
            return data["best_individual"], data["best_fitness"]
    except FileNotFoundError:
        return None, None

In [ ]:
import pygame
from pygame.locals import *
import random
import itertools
from capta import generate_population
from genetic_algorithm import mutate, order_crossover, generate_random_population, calculate_fitness, sort_population, default_problems
from draw_functions import draw_paths, draw_plot, draw_cities
import sys
import numpy as np
from benchmark_att48 import *




# Define constant values
# pygame
WIDTH, HEIGHT = 800, 400
NODE_RADIUS = 10
FPS = 30
PLOT_X_OFFSET = 450

# GA
N_CITIES = 15
POPULATION_SIZE = 100
N_GENERATIONS = 1000
MUTATION_PROBABILITY = 0.5

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)


# Initialize problem
# Using Random cities generation
# cities_locations = [(random.randint(NODE_RADIUS + PLOT_X_OFFSET, WIDTH - NODE_RADIUS), random.randint(NODE_RADIUS, HEIGHT - NODE_RADIUS))
#                     for _ in range(N_CITIES)]


# # Using Deault Problems: 10, 12 or 15
WIDTH, HEIGHT = 800, 400
cities_locations = default_problems[15]


# Using att48 benchmark
# WIDTH, HEIGHT = 1500, 800
# att_cities_locations = np.array(att_48_cities_locations)
# max_x = max(point[0] for point in att_cities_locations)
# max_y = max(point[1] for point in att_cities_locations)
# scale_x = (WIDTH - PLOT_X_OFFSET - NODE_RADIUS) / max_x
# scale_y = HEIGHT / max_y
# cities_locations = [(int(point[0] * scale_x + PLOT_X_OFFSET),
#                      int(point[1] * scale_y)) for point in att_cities_locations]
# target_solution = [cities_locations[i-1] for i in att_48_cities_order]
# fitness_target_solution = calculate_fitness(target_solution)
# print(f"Best Solution: {fitness_target_solution}")
# ----- Using att48 benchmark


# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("TSP Solver using Pygame")
clock = pygame.time.Clock()
generation_counter = itertools.count(start=1)  # Start the counter at 1



# Create Initial Population
# TODO:- use some heuristic like Nearest Neighbour or Convex Hull to initialize
# population = generate_random_population(cities_locations, POPULATION_SIZE)
population = generate_population(cities_locations, POPULATION_SIZE)
best_fitness_values = []
best_solutions = []

best_individual_ever, best_fitness_ever = load_best_individual()

# if best_individual_ever:
#     population[0] = best_individual_ever

# print(population)

# métade da inicilização aleattório e metade nearest neighbour



# Main game loop
running = True
best_fitness_ever = float('inf')
best_individual_ever = None

while running:
    for event in pygame.event.get():
        print(event)
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                running = False

    generation = next(generation_counter)
    
    if generation == N_GENERATIONS:
        running = False 

    screen.fill(WHITE)

    population_fitness = [calculate_fitness(
        individual) for individual in population]

    population, population_fitness = sort_population(
        population,  population_fitness)
    

    best_fitness = calculate_fitness(population[0])
    best_solution = population[0]

    best_fitness_values.append(best_fitness)
    best_solutions.append(best_solution)

    draw_plot(screen, list(range(len(best_fitness_values))),
              best_fitness_values, y_label="Fitness - Distance (pxls)")

    draw_cities(screen, cities_locations, RED, NODE_RADIUS)
    draw_paths(screen, best_solution, BLUE, width=3)
    draw_paths(screen, population[1], rgb_color=(128, 128, 128), width=1)

    # print(f"Generation {generation}: Best fitness = {round(best_fitness, 2)}")
    
     # Save the best individual if it is better than the one saved
    if best_fitness < best_fitness_ever:
        best_fitness_ever = best_fitness
        best_individual_ever = best_solution
        save_best_individual(best_individual_ever, best_fitness_ever)

    new_population = [population[0]]  # Keep the best individual: ELITISM
    
   

    while len(new_population) < POPULATION_SIZE:

        # selection
        # simple selection based on first 10 best solutions
        # parent1, parent2 = random.choices(population[:10], k=2)

        # solution based on fitness probability
        # probability = 1 / np.array(population_fitness)
        # parent1, parent2 = random.choices(population, weights=probability, k=2)
        
        # método de torneio

        parent1, parent2 = tournament_selection(population, population_fitness)

        # child1 = order_crossover(parent1, parent2)
        child1 = order_crossover(parent1, parent1)

        child1 = mutate(child1, MUTATION_PROBABILITY)

        new_population.append(child1)

    population = new_population

    pygame.display.flip()
    clock.tick(FPS)


# TODO: save the best individual in a file if it is better than the one saved.



# exit software
pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.1)
Hello from the pygame community. https://www.pygame.org/contribute.html
<Event(4352-AudioDeviceAdded {'which': 0, 'iscapture': 0})>
<Event(4352-AudioDeviceAdded {'which': 1, 'iscapture': 0})>
<Event(4352-AudioDeviceAdded {'which': 2, 'iscapture': 0})>
<Event(4352-AudioDeviceAdded {'which': 3, 'iscapture': 0})>
<Event(4352-AudioDeviceAdded {'which': 4, 'iscapture': 0})>
<Event(4352-AudioDeviceAdded {'which': 0, 'iscapture': 1})>
<Event(4352-AudioDeviceAdded {'which': 1, 'iscapture': 1})>
<Event(32774-WindowShown {'window': None})>
<Event(32770-VideoExpose {})>
<Event(32776-WindowExposed {'window': None})>
saved
saved


c:\Users\kake\Desktop\Pós\Fase 2\assets\draw_functions.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(4, 4), dpi=100)
